In [ ]:
import numpy as np
from scipy.stats import multivariate_normal
from scipy.optimize import linear_sum_assignment
from scipy.linalg import sqrtm
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

# --- Parameters ---
N = 100  # number of agents
K = 2    # number of GMM components

# Generate initial GMM (same for all agents)
mu0 = np.array([0.0, 0.0])
Sigma0 = np.array([[1.0, 0.3], [0.3, 1.0]])
X_init = np.random.multivariate_normal(mu0, Sigma0, N)

# Define target GMM parameters
weights = np.array([0.4, 0.6])
mu_targets = np.array([[5.0, 0.0],
                       [0.0, 5.0]])
Sigma_targets = np.array([
    [[1.0, 0.0], [0.0, 0.5]],
    [[0.5, 0.0], [0.0, 1.0]]
])

# --- Step 1: Compute cost matrix ---
cost_matrix = np.zeros((N, K))
for i in range(N):
    for k in range(K):
        diff = X_init[i] - mu_targets[k]
        Sigma_inv = np.linalg.inv(Sigma_targets[k])
        cost_matrix[i, k] = diff.T @ Sigma_inv @ diff

# --- Step 2: Exact assignment ---
n_k = (weights * N).astype(int)
assert np.sum(n_k) == N, "Weights must sum to 1 and N divisible accordingly."

target_labels = []
for k in range(K):
    target_labels += [k] * n_k[k]

cost_expanded = np.zeros((N, N))
for i in range(N):
    for j in range(N):
        k = target_labels[j]
        cost_expanded[i, j] = cost_matrix[i, k]

row_ind, col_ind = linear_sum_assignment(cost_expanded)
assigned_groups = [target_labels[j] for j in col_ind]

# --- Step 3: Transport maps ---
def affine_transport(x, mu_src, Sigma_src, mu_tgt, Sigma_tgt):
    A = sqrtm(Sigma_tgt) @ np.linalg.inv(sqrtm(Sigma_src))
    return A @ (x - mu_src) + mu_tgt

X_goal = np.zeros_like(X_init)
for i in range(N):
    k = assigned_groups[i]
    X_goal[i] = affine_transport(X_init[i], mu0, Sigma0, mu_targets[k], Sigma_targets[k])

# --- Step 4: Trajectories ---
def interpolate(x_start, x_end, steps=20):
    return np.linspace(x_start, x_end, steps)

trajectories = np.array([interpolate(X_init[i], X_goal[i]) for i in range(N)])

# --- Gaussian plotting helper ---
def plot_gaussian(ax, mean, cov, color='black', label=None, n_std=1.0):
    """Plot an n-std confidence ellipse of a 2D Gaussian."""
    vals, vecs = np.linalg.eigh(cov)  # eigen decomposition
    order = vals.argsort()[::-1]
    vals = vals[order]
    vecs = vecs[:, order]
    
    # Width and height are 2*n_std*sqrt(eigenvalue)
    width, height = 2 * n_std * np.sqrt(vals)
    
    # Angle between x-axis and the largest eigenvector
    angle = np.degrees(np.arctan2(*vecs[:, 0][::-1]))
    
    ellipse = Ellipse(xy=mean, width=width, height=height,
                      angle=angle, edgecolor=color, fc='None', lw=2, label=label)
    ax.add_patch(ellipse)


# --- Plot everything ---
fig, ax = plt.subplots(figsize=(7, 7))

# Plot trajectories
color_map = ['blue', 'green']
for i in range(N):
    k = assigned_groups[i]
    traj = trajectories[i]
    ax.plot(traj[:, 0], traj[:, 1], color=color_map[k], alpha=0.5)
    ax.scatter(traj[0, 0], traj[0, 1], color=color_map[k], s=8)   # start position
    ax.scatter(traj[-1, 0], traj[-1, 1], color=color_map[k], s=20, marker='x')  # goal position

# Plot start Gaussian
plot_gaussian(ax, mu0, Sigma0, color='black', label='Start Gaussian')

# Plot target Gaussians
for k in range(K):
    plot_gaussian(ax, mu_targets[k], Sigma_targets[k], color=color_map[k], label=f'Target {k+1}')

ax.set_aspect('equal')
ax.legend()
ax.set_title("Agent Trajectories with GMM Assignment and Gaussian Contours")
plt.grid(True)
plt.show()

